In [18]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import sqlite3

In [2]:
# monta o google drive corretamente
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
conn = sqlite3.connect('/content/drive/MyDrive/data_japan.db')
cur = conn.cursor()

In [4]:
# Carregar planilhas
from google.colab import files
uploaded = files.upload()

In [5]:
# pandas tables
df_immi_entrance = pd.read_csv('entrance in japan - nationalities_year.csv')
df_foreigner_country = pd.read_csv('Foreigners Residing in Japan by Nationality and Region by Status of Residence 2024 data(Country).csv')

In [6]:
# sql tables
df_immi_entrance.to_sql('immi_entrance', conn, if_exists='replace', index=False)
df_foreigner_country.to_sql('foreigner_country', conn, if_exists='replace', index=False)

6

## Immigration by Prefecture
#### 2023/24 Data

1. What are the top 10 immigrant nationalities entering Japan in 2023?

In [7]:
df_immi_entrance.head()

,year,country,total,diplomacy,government business,professor,art,religion,news_report,Investment,...,studying_abroad,training,family_stay,specific_activities,permanent_resident,spouses_of_japanese_other,spouses_permanent_residents,settlers,special_permanent_resident,Special Permanent Resident
0,2023,Asia,20549248,5620,22004,8771,447,3396,184,***,...,15423772,332948,5735,191942,39752,564307,91371,30198,70742,69171
1,2023,Afghanistan,3333,13,6,9,0,0,0,***,...,144,110,0,1059,52,124,22,17,50,0
2,2023,U.A.E,10280,115,8,0,0,0,0,***,...,9934,150,0,11,0,7,3,0,0,0
3,2023,Myanmar,47940,25,198,76,1,16,0,***,...,1799,8172,35,776,421,1113,308,89,693,0
4,2023,Bahrain,1014,21,23,0,0,0,0,***,...,475,14,0,0,0,1,5,0,0,0


In [12]:
# top 10 countries entrance in Japan in 2023 , excluding continents
pd.read_sql('''
                SELECT country, total
                FROM immi_entrance
                WHERE year = 2023
                AND country NOT IN ('Asia', 'Europe', 'South America', 'North America', 'Africa', 'Oceania')
                ORDER BY total DESC
                LIMIT 10;
                ''', conn)

,country,total
0,Korea,7144615
1,Taiwan,4173929
2,China,2678102
3,United States,2082697
4,China [Hong Kong],2049954
5,Thailand,1020690
6,Philippines,734011
7,Australia,620480
8,Vietnam,602337
9,Singapore,593997


2. What are the top 5 nationalities residing in Japan?

In [49]:
# replacing the ',' in total, then converting total to numeric type
pd.read_sql('''
                SELECT "Nationality / Region",
                       CAST(REPLACE(total, ',', '') AS INTEGER) as total_numeric
                FROM foreigner_country
                ORDER BY total_numeric DESC
                LIMIT 5;
''', conn)

,Nationality / Region,total_numeric
0,China,844187
1,Vietnam,600348
2,Korea,411043
3,Philippines,332293
4,Brazil,212325


2. What are the most common status of residence for each nationalities?
How are these groups distributed across the country's prefectures?

Displayed in an interactive Dashboard in [Tableau](https://public.tableau.com/app/profile/gabriela.yoshizaki/viz/WorkHoursandHourlyWageinJapanbyGender/Dashboard2)